In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATv2Conv

from GraphDataset import MyDataset
from torch_geometric.loader import DataLoader

torch.manual_seed(611)

In [2]:
# Load soft labels with membership level to each cover

df_data = pd.read_csv('data/ReyZamuro_softLabels.csv',index_col=0)
df_data = df_data.drop('RZUB02')
df_data = df_data.drop('RZUA03b')
print(df_data.head())

                   1         2         3
field_numb                              
RZUA01      1.000000  0.000000  0.000000
RZUA02      1.000000  0.000000  0.000000
RZUA04      0.589278  0.410722  0.000000
RZUA05      0.892664  0.000000  0.107336
RZUA06      0.938423  0.000000  0.061577


In [3]:
#Create list of ARUs and labels

DatosN = list(df_data.index)
Clases = df_data.values#.argmax(1)
etiquetasN = torch.tensor(Clases)

In [4]:
features = 'YAMNet'#'PANNs'#'YAMNet' #'VGGish'#'AI'

train_dataset = MyDataset(ListaArchivos=DatosN,
                          etiquetas=etiquetasN, caract=features)
print(len(train_dataset))

92


In [5]:
unpacked_data = [train_dataset[i][0] for i in range(len(train_dataset))]
x = torch.stack(unpacked_data, dim=0).transpose(0,1)

RZUA01
Data1.shape=torch.Size([3, 24, 1024])
RZUA02
Data1.shape=torch.Size([3, 24, 1024])
RZUA04
Data1.shape=torch.Size([3, 24, 1024])
RZUA05
Data1.shape=torch.Size([3, 24, 1024])
RZUA06
Data1.shape=torch.Size([3, 24, 1024])
RZUA07
Data1.shape=torch.Size([3, 24, 1024])
RZUA08
Data1.shape=torch.Size([3, 24, 1024])
RZUA09
Data1.shape=torch.Size([3, 24, 1024])
RZUA10
Data1.shape=torch.Size([3, 24, 1024])
RZUB01
Data1.shape=torch.Size([3, 24, 1024])
RZUB03
Data1.shape=torch.Size([3, 24, 1024])
RZUB04
Data1.shape=torch.Size([3, 24, 1024])
RZUB05
Data1.shape=torch.Size([3, 24, 1024])
RZUB06
Data1.shape=torch.Size([3, 24, 1024])
RZUB07
Data1.shape=torch.Size([3, 24, 1024])
RZUB08
Data1.shape=torch.Size([3, 24, 1024])
RZUB09
Data1.shape=torch.Size([3, 24, 1024])
RZUB10
Data1.shape=torch.Size([3, 24, 1024])
RZUB11
Data1.shape=torch.Size([3, 24, 1024])
RZUC01
Data1.shape=torch.Size([3, 24, 1024])
RZUC02
Data1.shape=torch.Size([3, 24, 1024])
RZUC03
Data1.shape=torch.Size([3, 24, 1024])
RZUC05
Dat

In [6]:
def min_max_normalize_along_dim(tensor, dim):
    # Get min and max values along the specified dimension, keeping dimensions
    min_vals, _ = torch.min(tensor, dim=dim, keepdim=True)
    max_vals, _ = torch.max(tensor, dim=dim, keepdim=True)
    
    # Handle the case where min and max are the same (to avoid division by zero)
    diff = max_vals - min_vals
    diff[diff == 0] = 1.0  # Replace zeros with ones to avoid division by zero
    
    # Normalize the tensor
    normalized = (tensor - min_vals) / diff
    
    return normalized

In [7]:
x = min_max_normalize_along_dim(x, dim=2)
x.shape

torch.Size([3, 92, 24, 1024])

## Edge Creation

In [8]:
from utils import edge_creation_nodeinfo, is_connected, edge_creation_geoDistance, plot_distance_matrix_heatmap, edge_creation_coverinfo
from torch_geometric.utils import is_undirected

In [9]:
graphs = edge_creation_coverinfo(torch.tensor(df_data.values), x,'knn', k_neigh=5)
for i in graphs:
    print(f"Is the graph {i} connected? {is_connected(i)}")
    print(f'Is the graph undirected {is_undirected(i.edge_index)}')

max_nodes = max([data.num_nodes for data in graphs])
num_feat = graphs[0].x.shape[-1]
num_feat

Is the graph Data(x=[92, 24, 1024], edge_index=[2, 606]) connected? True
Is the graph undirected True
Is the graph Data(x=[92, 24, 1024], edge_index=[2, 606]) connected? True
Is the graph undirected True
Is the graph Data(x=[92, 24, 1024], edge_index=[2, 606]) connected? True
Is the graph undirected True


1024

In [10]:
train_loader = DataLoader(graphs, batch_size=1, shuffle=False)

In [11]:
del graphs, x, unpacked_data, train_dataset

## Crear modelo y entrenar

In [12]:
from models import MatrixGCNVAE_MT, MatrixGATVAE_MT

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if features =='PANNs' or features == "YAMNet":
    device = torch.device('cpu')
print(device)

cpu


In [14]:
etiquetasN = etiquetasN.to(device)

In [15]:
model = MatrixGCNVAE_MT(in_channels=64*num_feat, hidden_channels=4*num_feat, latent_dim=int(num_feat/8), num_nodes=max_nodes, num_classes=3)
model = model.to(device)
etiquetasN = etiquetasN.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #0.01 GCN

# Training function
def train():
    model.train()
    model.double()

    total_loss = 0
    total_feature_loss = 0
    total_adj_loss = 0
    total_kl_loss = 0
    total_class_loss = 0
    total_accuracy = 0
    graph_count = 0


    for graph in train_loader:
        graph = graph.to(device)

        optimizer.zero_grad()
        node_reconstruction, adj_reconstruction, mu, logvar, class_logits = model(graph.x.double(), graph.edge_index)
        loss, feature_loss, adj_loss, kl_loss, class_loss = model.loss_function(node_reconstruction, adj_reconstruction, class_logits, etiquetasN, graph.x.double(),
                                                                    graph.edge_index, mu, logvar, alpha=1.0, beta=0.1)
        loss.backward()
        optimizer.step()
        accuracy = F.softmax(class_logits, dim=1).argmax(dim=1).eq(etiquetasN.argmax(dim=1)).sum()

        # Acumular métricas si deseas promediarlas después
        total_loss += loss.item()
        total_feature_loss += feature_loss.item()
        total_adj_loss += adj_loss.item()
        total_kl_loss += kl_loss.item()
        total_class_loss += class_loss.item()
        total_accuracy += accuracy.item()
        graph_count += 1
        # print(f'{F.softmax(class_logits, dim=1).argmax(dim=1)=}, {etiquetasN.argmax(dim=1)=}')
    avg_loss = total_loss / graph_count
    avg_feature_loss = total_feature_loss / graph_count
    avg_adj_loss = total_adj_loss / graph_count
    avg_kl_loss = total_kl_loss / graph_count
    avg_class_loss = total_class_loss / graph_count
    avg_accuracy = total_accuracy / graph_count
    
    return avg_loss, avg_feature_loss, avg_adj_loss, avg_kl_loss, avg_class_loss, avg_accuracy

# Train the model

# Configuración para early stopping
patience = 40  # Número de épocas a esperar para una mejora
best_loss = float('inf')
counter = 0
early_stop = False

# Para guardar todas las métricas
all_metrics = []

for epoch in range(1, 1001):
    loss, feature_loss, adj_loss, kl_loss, class_loss, accuracy = train()

    # Guardar métricas de la época actual
    metrics = {
        'epoch': epoch,
        'loss': loss,
        'feature_loss': feature_loss,
        'adj_loss': adj_loss,
        'kl_loss': kl_loss,
        'class_loss': class_loss,
        'accuracy': accuracy
    }
    all_metrics.append(metrics)

    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'====> Adj loss: {adj_loss:.4f}, '
          f'Recon: {feature_loss:.4f}, KL: {kl_loss:.4f}, Class: {class_loss:.4f}, '
          f'Accuracy: {accuracy:.2f}%')
        
    # Early stopping y guardado del mejor modelo
    if loss < best_loss:
        best_loss = loss
        counter = 0
        torch.save(model.state_dict(), f'modelsGVAE_semi/{features}_GCN_model_weights.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping activado en la época {epoch}")
            early_stop = True
    
    # Si se activa early stopping, salir del bucle
    if early_stop:
        break

# Guardar todas las métricas en un CSV
metrics_df = pd.DataFrame(all_metrics)
csv_path = f'modelsGVAE_semi/training_metrics_{features}_GCN_model.csv'
metrics_df.to_csv(csv_path, index=False)
print(f"Métricas guardadas en {csv_path}")

Epoch: 020, Loss: 1.9383
====> Adj loss: 0.9284, Recon: 0.0328, KL: 0.1344, Class: 0.8427, Accuracy: 65.33%
Epoch: 040, Loss: 1.8926
====> Adj loss: 0.9284, Recon: 0.0315, KL: 0.1808, Class: 0.7518, Accuracy: 71.67%
Epoch: 060, Loss: 1.9119
====> Adj loss: 0.9284, Recon: 0.0316, KL: 0.1678, Class: 0.7841, Accuracy: 69.67%
Epoch: 080, Loss: 1.8754
====> Adj loss: 0.9284, Recon: 0.0313, KL: 0.2049, Class: 0.7108, Accuracy: 75.00%
Epoch: 100, Loss: 1.8698
====> Adj loss: 0.9284, Recon: 0.0314, KL: 0.1799, Class: 0.7302, Accuracy: 73.33%
Epoch: 120, Loss: 1.8942
====> Adj loss: 0.9284, Recon: 0.0314, KL: 0.1766, Class: 0.7579, Accuracy: 73.33%
Epoch: 140, Loss: 1.8899
====> Adj loss: 0.9284, Recon: 0.0314, KL: 0.1654, Class: 0.7647, Accuracy: 75.00%
Epoch: 160, Loss: 1.8311
====> Adj loss: 0.9284, Recon: 0.0313, KL: 0.1489, Class: 0.7224, Accuracy: 78.67%
Epoch: 180, Loss: 1.8305
====> Adj loss: 0.9284, Recon: 0.0312, KL: 0.1686, Class: 0.7023, Accuracy: 79.67%
Epoch: 200, Loss: 1.8021
===

In [ ]:
model = MatrixGATVAE_MT(in_channels=64*num_feat, hidden_channels=4*num_feat, latent_dim=int(num_feat/8), num_nodes=max_nodes, num_classes=3)
model = model.to(device)
etiquetasN = etiquetasN.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #0.01 GCN

# Training function
def train():
    model.train()
    model.double()

    total_loss = 0
    total_feature_loss = 0
    total_adj_loss = 0
    total_kl_loss = 0
    total_class_loss = 0
    total_accuracy = 0
    graph_count = 0


    for graph in train_loader:
        graph = graph.to(device)

        optimizer.zero_grad()
        node_reconstruction, adj_reconstruction, mu, logvar, class_logits = model(graph.x.double(), graph.edge_index)
        loss, feature_loss, adj_loss, kl_loss, class_loss = model.loss_function(node_reconstruction, adj_reconstruction, class_logits, etiquetasN, graph.x.double(),
                                                                    graph.edge_index, mu, logvar, alpha=1.0, beta=0.1)
        loss.backward()
        optimizer.step()
        accuracy = F.softmax(class_logits, dim=1).argmax(dim=1).eq(etiquetasN.argmax(dim=1)).sum()

        # Acumular métricas si deseas promediarlas después
        total_loss += loss.item()
        total_feature_loss += feature_loss.item()
        total_adj_loss += adj_loss.item()
        total_kl_loss += kl_loss.item()
        total_class_loss += class_loss.item()
        total_accuracy += accuracy.item()
        graph_count += 1
        # print(f'{F.softmax(class_logits, dim=1).argmax(dim=1)=}, {etiquetasN.argmax(dim=1)=}')
    avg_loss = total_loss / graph_count
    avg_feature_loss = total_feature_loss / graph_count
    avg_adj_loss = total_adj_loss / graph_count
    avg_kl_loss = total_kl_loss / graph_count
    avg_class_loss = total_class_loss / graph_count
    avg_accuracy = total_accuracy / graph_count
    
    return avg_loss, avg_feature_loss, avg_adj_loss, avg_kl_loss, avg_class_loss, avg_accuracy

# Train the model

# Configuración para early stopping
patience = 40  # Número de épocas a esperar para una mejora
best_loss = float('inf')
counter = 0
early_stop = False

# Para guardar todas las métricas
all_metrics = []

for epoch in range(1, 1001):
    loss, feature_loss, adj_loss, kl_loss, class_loss, accuracy = train()

    # Guardar métricas de la época actual
    metrics = {
        'epoch': epoch,
        'loss': loss,
        'feature_loss': feature_loss,
        'adj_loss': adj_loss,
        'kl_loss': kl_loss,
        'class_loss': class_loss,
        'accuracy': accuracy
    }
    all_metrics.append(metrics)

    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'====> Adj loss: {adj_loss:.4f}, '
          f'Recon: {feature_loss:.4f}, KL: {kl_loss:.4f}, Class: {class_loss:.4f}, '
          f'Accuracy: {accuracy:.2f}%')
        
    # Early stopping y guardado del mejor modelo
    if loss < best_loss:
        best_loss = loss
        counter = 0
        torch.save(model.state_dict(), f'modelsGVAE_semi/{features}_GAT_model_weights.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping activado en la época {epoch}")
            early_stop = True
    
    # Si se activa early stopping, salir del bucle
    if early_stop:
        break

# Guardar todas las métricas en un CSV
metrics_df = pd.DataFrame(all_metrics)
csv_path = f'modelsGVAE_semi/training_metrics_{features}_GAT_model.csv'
metrics_df.to_csv(csv_path, index=False)
print(f"Métricas guardadas en {csv_path}")

## Fin